# LangGraph를 사용한 고객 지원 챗봇을 구축

- 웹 검색을 통해 일반적인 질문에 답변 
- 대화 상태를 유지하여 연속적인 대화  
- 복잡한 질문을 사람이 검토하도록 라우팅  
- 사용자 지정 상태(Custom State)를 활용하여 챗봇의 동작 제어  
- 대화 흐름을 되돌리고(Rewind), 다른 대화 경로 탐색 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적 설정 활성화
os.environ["LANGSMITH_TRACING"] = "true"

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano")

먼저 모델을 직접 사용해 봅니다. `ChatModel`은 LangChain의 **"Runnable"** 인스턴스이며, 이는 표준화된 인터페이스를 통해 상호작용할 수 있음을 의미합니다.  

모델을 간단하게 호출하려면 `.invoke` 메서드에 **메시지 목록**을 전달하면 됩니다.

## 도구를 활용한 챗봇 강화**   
이제 챗봇이 **웹에서 관련 정보를 찾아 더 나은 답변을 제공할 수 있도록 개선**합니다. 

---
먼저 **Tavily 검색 엔진**을 사용하기 위해 필요한 패키지를 설치하고 **`TAVILY_API_KEY`** 를 설정 합니다.  

In [7]:
# %%capture --no-stderr
# %pip install -U langchain-tavily

In [8]:
# Tavily 검색 툴을 임포트
# Tavily는 웹 검색을 통해 정보를 가져오는 도구로, LangChain에서 도구(tool)로 활용 가능
from langchain_tavily import TavilySearch

# TavilySearch 툴 인스턴스를 생성 (최대 결과 2개로 제한)
tool = TavilySearch(max_results=2)

# 사용할 툴들을 리스트로 구성 (여러 개의 도구가 필요한 경우를 대비해 리스트 형태로 작성)
tools = [tool]

# Tavily 검색 도구를 직접 호출하여 "LangGraph에서 node가 뭐야?"라는 질문에 대한 웹 검색 결과를 가져옴
result = tool.invoke("LangGraph에서 node가 뭐야?")
print(result)

{'query': 'LangGraph에서 node가 뭐야?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://dev-studyingblog.tistory.com/112', 'title': 'LangGraph란? - Dev studying blog - 티스토리', 'content': '노드(Node). LangGraph에서 노드는 개별 작업을 수행하는 단위이다. 예를 들어 프롬프트 처리 노드, AI 모델 실행 노드, 응답 요약 노드 등을 만들 수', 'score': 0.9202254, 'raw_content': None}, {'url': 'https://wikidocs.net/261580', 'title': '1-3-3. 노드 (Node) - LangGraph 가이드북 - 위키독스', 'content': '노드의 개념. 노드는 LangGraph에서 실제 작업을 수행하는 단위입니다. 각 노드는 특정 기능을 수행하는 Python 함수로 구현됩니다.', 'score': 0.9057319, 'raw_content': None}], 'response_time': 1.32}


In [10]:
# pip install -U "langchain[openai]"

### 에이전트 생성하기
이제 도구들과 LLM(언어 모델)을 정의했으니, 에이전트를 생성할 수 있습니다. 우리는 **LangGraph**를 사용하여 에이전트를 구성할 것입니다. 현재는 **상위 수준의 인터페이스**를 사용하여 에이전트를 만들고 있지만, LangGraph의 장점은 이 상위 수준 인터페이스가 **하위 수준의 고도로 제어 가능한 API**로 지원된다는 점입니다. 따라서 나중에 에이전트 로직을 자유롭게 수정할 수 있습니다.

에이전트는 세 가지 구성 요소로 이루어져 있습니다:
**대규모 언어 모델(LLM)**, 사용할 수 있는 **도구들의 집합**, 그리고 **지시사항을 담은 프롬프트**입니다.

LLM은 루프 방식으로 작동합니다. 각 반복(iteration)마다 다음과 같은 과정을 수행합니다:

1. 사용할 도구를 선택하고,
2. 그 도구에 입력을 제공하며,
3. 결과(관찰값, observation)를 받아오고,
4. 그 관찰값을 바탕으로 다음 행동을 결정합니다.

이 루프는 **중지 조건**이 충족될 때까지 계속되며, 일반적으로는 **사용자에게 응답하기에 충분한 정보를 수집했을 때** 종료됩니다.

In [11]:
# Import relevant functionality
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [12]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "안녕, 난 길동이야. 지금 서울의 날씨가 어때?",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

안녕, 난 길동이야. 지금 서울의 날씨가 어때?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_l690IFvcmD7kioTSUrgmcXpo)
 Call ID: call_l690IFvcmD7kioTSUrgmcXpo
  Args:
    query: 서울 날씨
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "서울 날씨", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.accuweather.com/ko/kr/seoul/226081/weather-forecast/226081", "title": "서울특별시, 서울시, 대한민국 3일 날씨 예보 - AccuWeather", "content": "오늘 5\\. 12. Image 28 77° 55° 흐릿함 Image 29약간 흐림 0%화 5\\. 13. Image 30 79° 55° 대체로 흐림 Image 31맑음 3%수 5\\. 14. Image 32 80° 60° 흐릿함 Image 33대체로 흐림; 밤 늦게 때때로 강한 뇌우가 내림 1%목 5\\. 15. Image 34 70° 59° 오전에 거센 소나기; 대체로 흐림 Image 35대체로 흐림 81%금 5\\. 16. Image 36 78° 61° 흐리고 때때로 소나기가 내림 Image 37대체로 흐림 55%토 5\\. 17. Image 38 79° 61°

In [13]:
from langgraph.checkpoint.memory import MemorySaver

# 대화 상태를 메모리에 저장할 수 있는 체크포인터 생성
memory = MemorySaver()

# ReAct 기반 에이전트 실행기 생성 (모델, 도구, 체크포인터를 연결)
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# 사용자별 스레드 ID 등을 설정할 수 있는 구성(config)
config = {"configurable": {"thread_id": "abc123"}} 

# 사용자 입력 메시지 정의
input_message = {
    "role": "user",           
    "content": "안녕, 난 길동이야."  
}

# 에이전트를 스트리밍 방식으로 실행하고, 응답을 단계별로 출력
for step in agent_executor.stream(
    {"messages": [input_message]},  # 초기 메시지 목록
    config,                         # 구성 정보 (스레드 ID 등)
    stream_mode="values"           # 스트리밍 방식: 값(value)들만 출력
):
    # 각 단계별 응답 메시지 중 마지막 메시지를 보기 좋게 출력
    step["messages"][-1].pretty_print()

================================ Human Message =================================

안녕, 난 길동이야.
================================== Ai Message ==================================

안녕, 길동님! 어떻게 도와드릴까요?


### 무한 loop 로 Chatbot 구현

In [14]:
from langchain_core.messages import AIMessage

memory = MemorySaver()

# 그래프를 메모리 체크포인트와 함께 컴파일합니다.
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc345"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    for step in agent_executor.stream({"messages": [user_input]}, config, stream_mode="values"):
        step["messages"][-1].pretty_print()

User:  안녕 난 길동이야


================================ Human Message =================================

안녕 난 길동이야
================================== Ai Message ==================================

안녕 길동이! 오늘 어떠한 일로 도와줄까?


User:  지금 서울의 날씨가 어때?


================================ Human Message =================================

지금 서울의 날씨가 어때?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_Ena3L4AeHD7aST1Te4bT8BUF)
 Call ID: call_Ena3L4AeHD7aST1Te4bT8BUF
  Args:
    query: 서울 날씨
================================= Tool Message =================================
Name: tavily_search

{"query": "서울 날씨", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.accuweather.com/ko/kr/seoul/226081/weather-forecast/226081", "title": "서울특별시, 서울시, 대한민국 3일 날씨 예보 - AccuWeather", "content": "Refresh Page 구름 많음 기온 시간별 예보 오전 11시 69° 0%오후 12시 72° 0%오후 1시 75° 0%오후 2시 76° 0%오후 3시 76° 0%오후 4시 77° 0%오후 5시 75° 0%오후 6시 73° 0%오후 7시 71° 0%오후 8시 68° 0%오후 9시 65° 0%오후 10시 63° 0% 일별 예보 오늘 5. 77° 55° 흐릿함 약간 흐림 0%화 5. 80° 60° 흐릿함 대체로 흐림; 밤 늦게 때때로 강한 뇌우가 내림 1%목 5. 70° 59° 오전에 거센 소나기; 대체로 흐림 대체로 흐림 81%금 5.", "score": 0.70287734, "raw_content": null}, {"url": "htt

User:  quit


Goodbye!
